# Finetune ruGPT3Small on your data


## Install enviroment

In [2]:
%pip install urllib3==1.25.4

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 125 kB 2.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.1.1 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.0.0 which is incompatible.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 2.10 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.25.4 which is incompatible.
aws-sam-translator 1.38.0 requires jsonschema~=3.2, but you have jsonschema 4.2.1 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [4]:
%pip install transformers==2.8.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 563 kB 2.5 MB/s 
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2021-11-25 15:16:30--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0.004s  

2021-11-25 15:16:31 (7.90 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [6]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2021-11-25 15:16:32--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-11-25 15:16:32 (110 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



In [3]:
#!g1.1
%pip install --ignore-installed torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 -f https://download.pytorch.org/whl/cu101/torch_stable.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 735.4 MB 2.6 kB/s 
     |████████████████████████████████| 12.8 MB 25.3 MB/s 
     |████████████████████████████████| 7.6 MB 2.8 MB/s 
     |████████████████████████████████| 15.7 MB 58.3 MB/s 
     |████████████████████████████████| 3.1 MB 59.5 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.8 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.0.0 which is incompatible.
spacy 

In [7]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [8]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8639, done.
remote: Counting objects: 100% (726/726), done.
remote: Compressing objects: 100% (455/455), done.
remote: Total 8639 (delta 434), reused 460 (delta 259), pack-reused 7913
Receiving objects: 100% (8639/8639), 14.46 MiB | 1.02 MiB/s, done.
Resolving deltas: 100% (5835/5835), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:230: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.0.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Defaulting to user installation because normal site-packages is not writeable
Created temporary directory: /tmp/pip-ephem-wheel-cache-lv7a0vrg
Created temporary directory: /tmp/pip-req-tracker-3145jqe4
Initialized build tracking at /tmp/pip-req-tracker-3145jqe4
Created build tracker: /tmp/pip-req-tracker-3145jqe4
Entered build tracker: 

## Prepare data

In [10]:
import pandas as pd
import numpy as np
import random
random.seed(1234)
np.random.seed(1234)
data = pd.read_csv('nekdoru.csv').dropna()
# anek = pd.read_csv('anekdotyru.csv')
# data = pd.concat(data, anek)
data.head()

,Unnamed: 0,text,rate
0,0,"Было на Свете семь великих чудес, а на восьмом...",9
1,1,"В ответ на заявление Трампа, что наплыв иммигр...",10
2,2,Специально для геймеров православная церковь в...,9
3,3,Причуды модернизации в сельском хозяйстве. Дер...,4
4,4,Цветы не самый практичный подарок на 8 Марта. ...,6


In [11]:
data.shape

(138083, 3)

In [12]:
val_ind = random.sample(range(data.shape[0]), 1000)
train = ['<s>' + data.iloc[i]['text'] for i in range(len(data)) if i not in val_ind]
valid = ['<s>' + data.iloc[i]['text'] for i in range(len(data)) if i in val_ind]
markov = [data.iloc[i]['text'] for i in range(len(data))]

In [13]:
len(train), len(valid)

(137083, 1000)

In [14]:
with open("train.txt", "w") as file:
    file.write("</s>\n".join(train))

In [15]:
with open("valid.txt", "w") as file:
    file.write("</s>\n".join(valid))

In [35]:
with open("markov.txt", "w") as file:
    file.write("\n".join(markov))

## Run finetuning

In [26]:
#!g1.1
!python3 pretrain_transformers.py \
    --output_dir=rugpt3_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --line_by_line \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --save_steps=0 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 64 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 400 \
    --overwrite_output_dir

████▏                  | 751/2143 [02:57<05:31,  4.20it/s]
Epoch: 100%|█████████████████████████████████████| 5/5 [42:14<00:00, 506.98s/it]
11/25/2021 17:10:38 - INFO - __main__ -    global_step = 10715, average loss = 1.2456921051580996
11/25/2021 17:10:38 - INFO - __main__ -   Saving model checkpoint to rugtp3_model
11/25/2021 17:10:38 - INFO - transformers.configuration_utils -   Configuration saved in rugtp3_model/config.json
11/25/2021 17:10:47 - INFO - transformers.modeling_utils -   Model weights saved in rugtp3_model/pytorch_model.bin
11/25/2021 17:10:47 - INFO - transformers.configuration_utils -   loading configuration file rugtp3_model/config.json
11/25/2021 17:10:47 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 2,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_

## Check our model

In [60]:
!python3 generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=rugpt3_model/ \
    --prompt='<s>Интересное наблюдение: автомобилисты чаще всего' \
    --k=10 \
    --num_return_sequences=5 \
    --p=0.95 \
    --length=200

/home/jupyter/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
11/25/2021 17:47:08 - INFO - transformers.tokenization_utils -   Model name 'rugpt3_model/' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'rugpt3_model/' is a path, a model identifier, or url to a directory containing tokenizer files.
11/25/2021 17:47:08 - INFO - transformers.tokenization_utils -   Didn't find file rugpt3_model/added_tokens.json. We won't load it.
11/25/2021 17:47:08 - INFO - transformers.tokenization_utils -   loading file rugpt3_model/vocab.json
11/25/2021 17:47:08 - INFO - transformers.tokenization_utils -   loading file rugpt3_model/merges.txt
1

### Markov chains to generate input to GPT3 model

ref: https://github.com/jsvine/markovify

In [33]:
%pip install markovify

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 235 kB 3.6 MB/s 
  Created wheel for markovify: filename=markovify-0.9.3-py3-none-any.whl size=18601 sha256=3a30ce99a4c29ff4c485dad6fc3a29c16f15117b938c4ed9a6897bb9c331f547
  Stored in directory: /tmp/xdg_cache/pip/wheels/d9/f0/5b/748a27bdf2496bd4df51acb9442dae516efce507ff4849813e
Successfully built markovify
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [69]:
import markovify

# Get raw text as string.
with open("markov.txt") as f:
    text = f.read()

# Build the model
text_model = markovify.NewlineText(text, retain_original=False)
text_model = text_model.compile()

In [62]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tok = GPT2Tokenizer.from_pretrained("rugpt3_model")
model = GPT2LMHeadModel.from_pretrained("rugpt3_model")

'<s>Какой-то текст в туалете застрял у меня на языке, но я быстро его дочитал и понял почему.</Software...Список поддерживаемых версиями программ для чтения электронных книг: "Дурацкий перевод". И еще - это не оригинал? А что за бред?! Я же просил перевести как можно лучше! На каком сайте был сайт с оригиналами моих произведений!? Вот он правильный источник всех этих идиотов из ЖЭКов. Они даже знают где туалет находится по адресу Гоголя 45а офис 407 через дорогу от моего подъезда к метро Пушкинская если их туда послали работать там уборщицами или дворниками потому шо так никто никогда раньше сюда пешком ходить им лень было идти а тут такой бардак кругом творится да все время куда они ходят то ли просто мусор вывозят чтобы сэкономить денег нетрадиционной ориентации были бы деньги тратить будут теперь эти уроды работают ведь нифига себе нашли работу которые сидят без лифтов вообще никому ненужных сайтов о том какая сейчас очередь стоит под окнами домов стоят перед ними везде после ремонт

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


In [65]:
text = "<s>" + ' '.join(text_model.make_sentence().split()[:random.randint(2, 5)])
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(inpt, max_length=500, repetition_penalty=1.0, do_sample=True, top_k=10, top_p=0.95, temperature=1)
output = tok.decode(out[0], clean_up_tokenization_spaces=True)
output = output[: output.find('</s>')]
print(output)

<s>Так и есть, у нас в деревне на каждой улице есть магазин "Дуршлаг" с кондиционером.


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


In [71]:
text_model.make_sentence().split()[:random.randint(2, 5)]

['Мужик', 'читает', 'газету']

In [73]:
a = text_model.to_json()

In [77]:
with open("markov.json", "w") as f:
    f.write(a)